In [12]:
import sys
sys.path.append("..")
from webapp.utils.azure_utils import KeyVault, DataLake
import pandas as pd

In [2]:
# Connect to Storage Account:
vault = KeyVault(keyVaultName = "keyvaultdva2022")
storage_credential = vault.get_secret(secretName = "storagePrimaryKey")
storage = DataLake(account_name = "storageaccountdva", credential = storage_credential)
file_system = "energyhub"

In [5]:
files = storage.list_directory_contents(file_system, directory = "forecasting/results")

In [9]:
dfs = []

for file in files:
    df = storage.pandas_read(file_system, directory="/", file_name=file)
    dfs.append(df)

In [13]:
df = pd.concat(dfs)

In [16]:
available_buildings = df.building_id.unique()

In [19]:
len(available_buildings)

937

In [20]:
meta_data = storage.pandas_read(file_system, directory="data_original/metadata", file_name="metadata.csv")

In [23]:
all_buildings = meta_data.building_id.unique()

In [58]:
excluded = set(all_buildings).difference(set(available_buildings))

In [59]:
excluded = pd.DataFrame(excluded, columns=["building_id"], index = range(len(excluded))).sort_values(by = "building_id").reset_index(drop =  True)

In [61]:
excluded.to_excel("excluded.xlsx")